### [TODO] 코드 구현 : 모델 예측 결과 csv로 만들기

In [1]:
# 데이터 다운로드
#!wget –no-check-certificate 'https://docs.google.com/uc?export=download&id=1IVvuG3SMlarSSGmcliGFjq1fMxZtksE0' -O kaggle-kakr-housing-data.zip

# 다운로드 받은 zip파일 압축풀기
#!unzip -qq ./kaggle-kakr-housing-data.zip

import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#--------------------------------------------------------------------------------------------------------------------------------------

train_data_path = join('/Users/juwonkim/Desktop/네이버 부스트코스 AI 엔지니어/week2/kaggle-kakr-housing-data/train.csv')
sub_data_path = join('/Users/juwonkim/Desktop/네이버 부스트코스 AI 엔지니어/week2/kaggle-kakr-housing-data/test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

#--------------------------------------------------------------------------------------------------------------------------------------
data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)

#--------------------------------------------------------------------------------------------------------------------------------------
y = data['price']
del data['price']

train_len = len(data)
data = pd.concat((data, sub), axis=0)

sub_id = data['id'][train_len:]
del data['id']

data['date'] = data['date'].apply(lambda x : str(x[:6])).astype(int)

#--------------------------------------------------------------------------------------------------------------------------------------
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)

#--------------------------------------------------------------------------------------------------------------------------------------
y_log_transformation = np.log1p(y)

sub = data.iloc[train_len:, :] # 테스트 데이터
x = data.iloc[:train_len, :] # 학습데이터

print(x.shape)
print(sub.shape)

gboost = GradientBoostingRegressor(random_state=2023)
xgboost = xgb.XGBRegressor(random_state=2023)
lightgbm = lgb.LGBMRegressor(random_state=2023)

models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'},
          {'model':lightgbm, 'name':'LightGBM'}]

def get_cv_score(models):
    kfold = KFold(n_splits=5).get_n_splits(x.values)
    for m in models:
        CV_score = np.mean(cross_val_score(m['model'], X=x.values, y=y, cv=kfold))
        print(f"Model: {m['name']}, CV score:{CV_score:.4f}")

get_cv_score(models)

def AveragingBlending(models, x, y, sub_x):
    # 모델학습
    for m in models :
        m['model'].fit(x.values, y)

    # 모델예측
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])

    # 각 모델 에측의 평균을 return
    return np.mean(predictions, axis=1)

y_pred = AveragingBlending(models, x, y, sub)
print(len(y_pred))
y_pred

submission = pd.read_csv('/Users/juwonkim/Desktop/네이버 부스트코스 AI 엔지니어/week2/kaggle-kakr-housing-data/sample_submission.csv')
submission.head()

result = pd.DataFrame({
    'id' : sub_id,
    'price' : y_pred
})

result.head()

my_submission_path = '/Users/juwonkim/Desktop/네이버 부스트코스 AI 엔지니어/week2/kaggle-kakr-housing-data/submission.csv'

## 코드시작 ##
# 미션 코드 작성 : result를 'my_submission_path'란 이름의 csv로 저장해 주세요.

result.to_csv(my_submission_path, index=False)


## 코드종료 ##

(15035, 19)
(6468, 19)
Model: GradientBoosting, CV score:0.8609


: 

### [TODO] 코드 구현 : public score 확인하고, 제출 결과 공유하기

In [ ]:
## 코드시작 ##
# 미션 코드 작성 : 캐글에 결과를 제출하고, Public Score를 확인해보세요.
print("나의 Public Score 점수는 : ")
print(...)  # ... 에 점수를 기록해주세요.

## 코드종료 ##

ALL RIGHTS RESERVED. (C)NAVER Connect Foundation.